In [1]:
import pandas as pd

In [2]:
import requests, re
from bs4 import BeautifulSoup

In [3]:
#remove comments because it confuses parser
comm = re.compile("<!--|-->")
def make_soup(url):
    page = requests.get(url).text
    soup = BeautifulSoup(comm.sub('', page), 'html.parser')
    return soup

In [12]:
def get_college_data(url):
    data = {}
    soup = make_soup(url)
    meta = soup.find(id='meta')
    weight_div = meta.find('span', itemprop='weight')
    if weight_div is not None:
        data['wt'] = weight_div.get_text()
    height_div = meta.find('span', itemprop='height')
    if height_div is not None:
        data['ht'] = height_div.get_text()

    passing = soup.find('table', id='passing')
    last_row = passing.tbody.find_all('tr')[-1]
    data['conf'] = last_row.find_all('td')[1].get_text()
    tds = passing.tfoot.find_all('td')
    data['college_cmps'] = tds[5].get_text()
    data['college_atts'] = tds[6].get_text()
    data['college_yds'] = tds[8].get_text()
    data['college_tds'] = tds[11].get_text()

    rushing = soup.find('table', id='rushing').tfoot
    tds = rushing.find_all('td')
    data['college_rushAtts'] = tds[5].get_text()
    data['college_rushYds'] = tds[6].get_text()
    data['college_rushTds'] = tds[8].get_text()

    return data

In [5]:
DRAFT_URL = 'https://www.pro-football-reference.com/play-index/draft-finder.cgi?request=1&year_min=1997&year_max=2017&draft_slot_min=1&draft_slot_max=500&pick_type=overall&pos%5B%5D=qb&conference=any&show=all&order_by=default'
page = requests.get(DRAFT_URL).text
soup = BeautifulSoup(page, 'html.parser')

In [6]:
table = soup.find(id='results')

In [7]:
NFL_STAT_NAMES = ['Year','Rnd','Pick','Player','Pos','DrAge','Tm','From','To','AP1','PB','St','CarAV','G','GS','QBrec','Cmp','Att','Yds','TD','Int','RushAtt','RushYds','RushTD','College','Link']
data = []

In [19]:
data = []
for tr in table.tbody.find_all('tr', class_=lambda x: x != 'thead'):
    player = {}
    for i, td in enumerate(tr.find_all('td')):
        if i != 25 or td.a is None:
            player[NFL_STAT_NAMES[i]] = td.get_text()
        else:
            print('gathering data for: ' + player['Player'])
            player.update(get_college_data(td.a['href']))
    data.append(player)

df = pd.DataFrame(data)
df

gathering data for: Mitchell Trubisky
gathering data for: Patrick Mahomes
gathering data for: Deshaun Watson
gathering data for: DeShone Kizer
gathering data for: Davis Webb
gathering data for: C.J. Beathard
gathering data for: Joshua Dobbs
gathering data for: Nathan Peterman
gathering data for: Brad Kaaya
gathering data for: Chad Kelly
gathering data for: Jared Goff
gathering data for: Carson Wentz
gathering data for: Paxton Lynch
gathering data for: Christian Hackenberg
gathering data for: Jacoby Brissett
gathering data for: Cody Kessler
gathering data for: Connor Cook
gathering data for: Dak Prescott
gathering data for: Cardale Jones
gathering data for: Kevin Hogan
gathering data for: Nate Sudfeld
gathering data for: Jake Rudock
gathering data for: Brandon Allen
gathering data for: Jeff Driskel
gathering data for: Brandon Doughty
gathering data for: Jameis Winston
gathering data for: Marcus Mariota
gathering data for: Garrett Grayson
gathering data for: Sean Mannion
gathering data f

,AP1,Att,CarAV,Cmp,College,DrAge,From,G,GS,Int,...,college_atts,college_cmps,college_rushAtts,college_rushTds,college_rushYds,college_tds,college_yds,conf,ht,wt
0,0,651,7,406,North Carolina,23,2017,22,22,16,...,572,386,120,8,439,41,4762,ACC,6-2,222lb
1,0,426,1,286,Texas Tech,21,2017,12,12,11,...,1349,857,308,22,845,93,11252,Big 12,6-3,230lb
2,0,513,7,327,Clemson,21,2017,17,16,17,...,1207,814,435,26,1934,90,10168,ACC,6-3,215lb
3,0,483,5,259,Notre Dame,21,2017,16,15,23,...,695,422,264,18,997,47,5805,Ind,6-4,233lb
4,0,,,,California,22,,,,,...,1367,841,77,9,-100,83,9852,Pac-12,6-5,229lb
5,0,393,4,225,Iowa,23,2017,13,10,13,...,782,454,224,10,429,40,5562,Big Ten,6-2,219lb
6,0,3,0,2,Tennessee,22,2018,4,0,0,...,999,614,438,32,2160,53,7138,SEC,6-3,210lb
7,0,130,1,68,Pittsburgh,23,2017,8,4,12,...,662,398,172,5,525,47,5236,ACC,6-2,225lb
8,0,,,,Miami (FL),22,,,,,...,1188,720,94,4,-386,69,9968,ACC,6-4,215lb
9,0,,0,,Mississippi,23,2018,1,0,,...,803,513,203,16,958,50,6858,SEC,6-2,224lb


In [23]:
df.dropna(subset=['college_atts'], inplace=True)

In [24]:
df.to_csv('data/scraped_data.csv')